In [1]:
!pip install wikidata
!pip install AST
!pip install SPARQLWrapper
!pip install pandas
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import pandas as pd
import numpy as np
import random
import json
import time
import datetime
import os

from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig

import torch
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup
from collections import Counter

In [6]:
import os
cwd = os.getcwd()
cwd

'/content'

In [7]:
train_fname = 'task1_wikidata_train.json'

pd.read_json(train_fname)

,id,question,category,type
0,0,What periodical literature does Delta Air Line...,resource,"[publication, recurring, intellectual work, te..."
1,1,Who is the child of Ranavalona I's husband?,resource,"[person, omnivore, natural person]"
2,2,Is it true Jeff_Bridges occupation Lane Chandl...,boolean,[boolean]
3,3,Which is the operating income for Qantas?,literal,[number]
4,4,which cola starts with the letter p,resource,"[soft drink, trademark, carbonated beverage, n..."
...,...,...,...,...
43549,54463,who developed outnumbered!,resource,[software company]
43550,54464,what is the sex of seumas o'kelly?,resource,"[gender, identity, sex]"
43551,54465,which position did herby fortunat play in foot...,resource,[position]
43552,54466,What is nikolaj frobenius's profession?,resource,"[job, specialty, occupation]"


In [8]:
train_fname = 'task1_wikidata_train.json'
wikidata_train_df = pd.read_json(train_fname)
print(len(wikidata_train_df))
wikidata_train_df = wikidata_train_df[wikidata_train_df.question.notna()]
print(len(wikidata_train_df))
wikidata_train_df= wikidata_train_df[wikidata_train_df.category.notna()]
print(len(wikidata_train_df))
wikidata_train_df= wikidata_train_df[wikidata_train_df.type.notna()]
print(len(wikidata_train_df))
wikidata_train_df.head()

43554
43554
43554
43554


,id,question,category,type
0,0,What periodical literature does Delta Air Line...,resource,"[publication, recurring, intellectual work, te..."
1,1,Who is the child of Ranavalona I's husband?,resource,"[person, omnivore, natural person]"
2,2,Is it true Jeff_Bridges occupation Lane Chandl...,boolean,[boolean]
3,3,Which is the operating income for Qantas?,literal,[number]
4,4,which cola starts with the letter p,resource,"[soft drink, trademark, carbonated beverage, n..."


In [9]:
wikidata_train_df['category'].unique()

array(['resource', 'boolean', 'literal'], dtype=object)

In [10]:
wikidata_train_df['category'].value_counts()

resource    36886
literal      4530
boolean      2138
Name: category, dtype: int64

In [11]:
wikidata_train_df = wikidata_train_df[(wikidata_train_df['category']!='numberstring')&(wikidata_train_df['category']!='stringnumber')&(wikidata_train_df['category']!='numberdate')&(wikidata_train_df['category']!='datenumber')]

In [12]:
for idx in wikidata_train_df.index:
    if wikidata_train_df['category'][idx] == 'literal':
        wikidata_train_df['category'][idx] = ''.join(wikidata_train_df['type'][idx])
classes = wikidata_train_df['category'].value_counts()
print(classes)
wikidata_train_df.head()

<ipython-input-12-324e44630a2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wikidata_train_df['category'][idx] = ''.join(wikidata_train_df['type'][idx])


resource        36886
boolean          2138
string           2090
date             1407
number           1006
numberstring       13
stringnumber        7
numberdate          6
datenumber          1
Name: category, dtype: int64


,id,question,category,type
0,0,What periodical literature does Delta Air Line...,resource,"[publication, recurring, intellectual work, te..."
1,1,Who is the child of Ranavalona I's husband?,resource,"[person, omnivore, natural person]"
2,2,Is it true Jeff_Bridges occupation Lane Chandl...,boolean,[boolean]
3,3,Which is the operating income for Qantas?,number,[number]
4,4,which cola starts with the letter p,resource,"[soft drink, trademark, carbonated beverage, n..."


In [13]:
label_map_dic = {
      'resource': 0,
      'boolean': 1,
      'string': 2,
      'number': 3,
      'date': 4
      }
wikidata_train_df['5classes'] = wikidata_train_df['category'].map(label_map_dic)
wikidata_train_df

,id,question,category,type,5classes
0,0,What periodical literature does Delta Air Line...,resource,"[publication, recurring, intellectual work, te...",0.0
1,1,Who is the child of Ranavalona I's husband?,resource,"[person, omnivore, natural person]",0.0
2,2,Is it true Jeff_Bridges occupation Lane Chandl...,boolean,[boolean],1.0
3,3,Which is the operating income for Qantas?,number,[number],3.0
4,4,which cola starts with the letter p,resource,"[soft drink, trademark, carbonated beverage, n...",0.0
...,...,...,...,...,...
43549,54463,who developed outnumbered!,resource,[software company],0.0
43550,54464,what is the sex of seumas o'kelly?,resource,"[gender, identity, sex]",0.0
43551,54465,which position did herby fortunat play in foot...,resource,[position],0.0
43552,54466,What is nikolaj frobenius's profession?,resource,"[job, specialty, occupation]",0.0


In [18]:
# Instantiate the Bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [19]:
input_ids = []
attention_masks = []
labels = []

MAX_LENGTH = 36

# For every sentence...
for i,row in wikidata_train_df.iterrows():
    # sent = str('[CLS]') + row['question'] + str('[SEP]')
    sent = row['question']
    labels.append(int(row['5classes']))
    encoded_dict = tokenizer.encode_plus(
                      sent,                          # Sentence to encode.
                      add_special_tokens = True,     # Add '[CLS]' and '[SEP]'
                      max_length = MAX_LENGTH,       # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,  # Construct attn. masks.
                      return_tensors = 'pt',         # Return pytorch tensors.
                  )
# print(encoded_dict)
  
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
  
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
print(labels.shape)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


torch.Size([43527])


In [ ]:
!pip install ipykernel

In [20]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

39,174 training samples
4,353 validation samples


In [21]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = torch.utils.data.DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = torch.utils.data.DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [22]:
model = BertForSequenceClassification.from_pretrained(
               'bert-base-uncased', 
               num_labels = len(label_map_dic), # The number of output labels. 2 for binary classification.  
               output_attentions = False,
               output_hidden_states = False)   

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# send model to device
model.to(device) 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [23]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 2,I have already seen that the model starts overfitting beyound 2 epochs
epochs = 5

# Total number of training steps is [number of batches] x [number of epochs]. 
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                       num_warmup_steps = 0, # Default value in run_glue.py
                       num_training_steps = total_steps)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [25]:
# Set the seed value all over the place to make this reproducible.
seed_val = 66

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# store a number of quantities such as training and validation loss,validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # training
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader andcopy each tensor to the GPU
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing abackward pass.
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, 
                token_type_ids=None, 
                attention_mask=b_input_mask, 
                labels=b_labels)

        # Accumulate the training loss over all of the batches
        loss= outputs[0]
        logits = outputs[1]  
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    
    # ========================================
    # Save model of every epoch
    output_dir = '../outputs/models_wikidata_final/'
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    models_dir = output_dir + 'BERT_model_10classes_epoch' + str(epoch_i + 1)
    # Create output directory if needed
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)

    print("Saving model to %s" % models_dir)

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(models_dir)
    tokenizer.save_pretrained(models_dir)
    
    # ========================================
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0.0
    total_eval_loss = 0.0
    nb_eval_steps = 0.0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        
            outputs= model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
            
        # Accumulate the validation loss.
        loss= outputs[0]
        logits = outputs[1] 
        
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.4f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

======== Epoch 1 / 5 ========
Training...
  Batch    40  of  1,225.    Elapsed: 0:00:11.
  Batch    80  of  1,225.    Elapsed: 0:00:20.
  Batch   120  of  1,225.    Elapsed: 0:00:28.
  Batch   160  of  1,225.    Elapsed: 0:00:37.
  Batch   200  of  1,225.    Elapsed: 0:00:46.
  Batch   240  of  1,225.    Elapsed: 0:00:56.
  Batch   280  of  1,225.    Elapsed: 0:01:05.
  Batch   320  of  1,225.    Elapsed: 0:01:15.
  Batch   360  of  1,225.    Elapsed: 0:01:24.
  Batch   400  of  1,225.    Elapsed: 0:01:32.
  Batch   440  of  1,225.    Elapsed: 0:01:41.
  Batch   480  of  1,225.    Elapsed: 0:01:50.
  Batch   520  of  1,225.    Elapsed: 0:01:59.
  Batch   560  of  1,225.    Elapsed: 0:02:08.
  Batch   600  of  1,225.    Elapsed: 0:02:17.
  Batch   640  of  1,225.    Elapsed: 0:02:27.
  Batch   680  of  1,225.    Elapsed: 0:02:36.
  Batch   720  of  1,225.    Elapsed: 0:02:45.
  Batch   760  of  1,225.    Elapsed: 0:02:54.
  Batch   800  of  1,225.    Elapsed: 0:03:03.
  Batch   840  of 

In [ ]:

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')
df_stats.to_csv('df_category_stats.csv',index=None)
df_stats

In [30]:
epoch = '5' 
models_dir = output_dir + 'BERT_model_5classes' + epoch

# Create output directory if needed
# if not os.path.exists(models_dir):
#     os.makedirs(models_dir)

print("Saving model to %s" % models_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(models_dir)
tokenizer.save_pretrained(models_dir)

Saving model to /content/outputBERT_model_5classes5


('/content/outputBERT_model_5classes5/tokenizer_config.json',
 '/content/outputBERT_model_5classes5/special_tokens_map.json',
 '/content/outputBERT_model_5classes5/vocab.txt',
 '/content/outputBERT_model_5classes5/added_tokens.json')

In [32]:
category_model = model

In [33]:
test_df = pd.read_json('/content/task1_wikidata_test.json')
print('test size: ',len(test_df))
test_df = test_df[test_df['question'].notna()]
print('After: ',len(test_df))

test size:  4355
After:  4355


In [34]:
def classify_category(q):
    input_ids = torch.tensor(category_tokenizer.encode(q, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1

    input_ids = input_ids.to(device) 
    labels = labels.to(device)
    with torch.no_grad():
        outputs = category_model(input_ids, labels=labels)
    logits = outputs[1]
    result = np.argmax(logits.cpu().numpy(),axis=1)[0]
    if result == 0:
        categoryLabel = 'resource'
    elif result == 1:
        categoryLabel = 'boolean'
    elif result == 2:
        categoryLabel = 'string'
    elif result == 3:
        categoryLabel = 'number'
    elif result == 4:
        categoryLabel = 'date'
    else:
        print('wrong prediction!!!')
    return categoryLabel

In [36]:
category_tokenizer = BertTokenizer.from_pretrained(models_dir)
category_model = BertForSequenceClassification.from_pretrained(models_dir,num_labels=5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# send model to device
category_model.to(device) 
print ('Success', device)
test_df['pred_category'] = test_df.apply(lambda x: classify_category(x['question']), axis=1)

Success cuda


In [37]:
test_df

,id,question,category,type,pred_category
0,10,Did Lindsey Vonn participate in the FIS Alpine...,boolean,[boolean],boolean
1,20,Is Kevin Costner owner of Fielders Stadium?,boolean,[boolean],boolean
2,48,What sovereign state replaced the Kingdom of G...,resource,"[state, former administrative territorial enti...",resource
3,59,What is established by Archbishop of Canterbur...,resource,"[person, omnivore, natural person]",resource
4,62,Is the throughput of the Vest-vassdraget equal...,boolean,[boolean],boolean
...,...,...,...,...,...
4350,54420,which artist recorded yummy yummy?,resource,[musical ensemble],resource
4351,54430,What gender is madeleine peters?,resource,"[gender, identity, sex]",resource
4352,54443,What genre of videogame is rumble roses?,resource,"[class, sport]",resource
4353,54448,which country was adventures of william tell m...,resource,"[political territorial entity, democracy, regi...",resource


## **Type Prediction**

In [25]:
import pandas as pd
import json
import csv

with open('hierarchy_recent.json', 'r') as f:
    hierarchy_data = json.load(f)
    

# Load training data
with open('task1_wikidata_train.json', 'r') as f:
    train_data = json.load(f)
    for item in train_data:
        train_question = item['question']  

# Load Qid label data
qid_labels = {}
with open('wikidata_qid_label.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # skip header row
    for row in reader:
        qid_labels[row[0]] = row[1]
        # print(qid_labels) 

results = {}

# key, item 
# value, best_match
for item in hierarchy_data:
    qid = list(item.keys())[0]
    level = item['level']
    children = item['children']
    
    # Find the closest matching qid label in the hierarchy
    best_match = None
    best_score = -1

    for child in children:
        if child in qid_labels:
            # print(children)
            # print(qid_labels)
            label = qid_labels[child]
            score = len(set(train_question.split()).intersection(label.split()))

            if score > best_score:

                best_match = child
                best_score = score
                
    results[qid] = best_match

    # Output the predicted answer type
    if best_match:
        print("Predicted answer type:", qid_labels[best_match])

print(results)

Predicted answer type: music
Predicted answer type: physical quantity
Predicted answer type: federative entity of Mexico
Predicted answer type: census place
Predicted answer type: pervasive game
Predicted answer type: protected name
Predicted answer type: social relation
Predicted answer type: regional standard
Predicted answer type: election in the United Kingdom
Predicted answer type: gender
Predicted answer type: silence
Predicted answer type: activist
Predicted answer type: animal organ
Predicted answer type: local administrative unit in Thailand
Predicted answer type: girls school
Predicted answer type: sailing ship
Predicted answer type: record label
Predicted answer type: puzzle
Predicted answer type: Kuiper belt object
Predicted answer type: former administrative territorial entity
Predicted answer type: religion
Predicted answer type: festival
Predicted answer type: square-free integer
Predicted answer type: elliptical galaxy
Predicted answer type: grotto
Predicted answer type

In [18]:
# Count the number of types
word_list = []
for word_arr in wikidata_train_df:
  for words in word_arr:
    if words not in word_list:
      word_list.append(words)

all_words_with_occurance = []
for word_arr in wikidata_train_df.type:
  for words in word_arr:
    all_words_with_occurance.append(words)

counted_words = Counter(all_words_with_occurance)
df = pd.DataFrame.from_dict(counted_words, orient='index').reset_index()
df = df.rename(columns={'index':'words', 0:'count'})

In [19]:
wiki_labels_csv = 'wikidata_qid_label.csv'
label_df = pd.read_csv(wiki_labels_csv)

# get the label and qid label
def wiki_label_and_id(label_df):
    
    label_to_id = {}
    id_to_label = {}

    for row in label_df.values:
        label_to_id[row[0]] = row[1]
        id_to_label[row[1]] = row[0]
    return label_to_id,id_to_label

label_to_id,id_to_label = wiki_label_and_id(label_df)